### Вычислить среднее количество ночей, которые гости проводят в отеле (только для подтвержденных бронирований, с детализацией по месяцам и годам)

In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

# Создание Spark сессии
spark = SparkSession.builder \
    .appName("Hotel - Farida Khamzina") \
    .getOrCreate()

# Шаг 1: Загрузка данных
logs_hotel_df = spark.read \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv("/content/Hotel.csv")

### Вычислить среднее количество ночей, которые гости проводят в отеле (только для подтвержденных бронирований, с детализацией по месяцам и годам)

In [21]:
result1 = (logs_hotel_df
    .filter(col("status") == "Not_Canceled")
    .withColumn("total_nights", col("weekend_nights") + col("week_nights"))
    .groupBy("year", "month")
    .agg(round(avg("total_nights"), 2).alias("avg_nights"))
    .orderBy("year", "month")
)
result1.show()

+----+-----+----------+
|year|month|avg_nights|
+----+-----+----------+
|2017|    7|      3.02|
|2017|    8|      2.72|
|2017|    9|      2.66|
|2017|   10|       2.7|
|2017|   11|      2.72|
|2017|   12|      3.04|
|2018|    1|      2.74|
|2018|    2|      2.69|
|2018|    3|      3.04|
|2018|    4|      2.92|
|2018|    5|      2.81|
|2018|    6|       2.6|
|2018|    7|      3.19|
|2018|    8|      3.15|
|2018|    9|      2.79|
|2018|   10|      2.89|
|2018|   11|      2.98|
|2018|   12|      3.25|
+----+-----+----------+



### Определить ТОП-3 месяца по проценту отмененных броней за 2018 год.

In [16]:
monthly_stats_2018 = (logs_hotel_df
    .filter(col("year") == 2018)
    .groupBy("year", "month")
    .agg(
        count("*").alias("total_bookings"),
        sum(when(col("status") == "Canceled", 1).otherwise(0)).alias("canceled_bookings")
    )
)

result2 = (monthly_stats_2018
    .withColumn("cancel_percentage",
                round((col("canceled_bookings") * 100.0 / col("total_bookings")), 2))
    .select("month", "total_bookings", "canceled_bookings", "cancel_percentage")
    .orderBy(col("cancel_percentage").desc())
    .limit(3)
)
result2.show()

+-----+--------------+-----------------+-----------------+
|month|total_bookings|canceled_bookings|cancel_percentage|
+-----+--------------+-----------------+-----------------+
|    8|          2799|             1303|            46.55|
|   10|          3404|             1578|            46.36|
|    9|          2962|             1356|            45.78|
+-----+--------------+-----------------+-----------------+



### Вычислить среднее время на каждый месяц между бронированием и заездом в отель для подтвержденных броней.

In [17]:
result3 = (logs_hotel_df
    .filter(col("status") == "Not_Canceled")
    .groupBy("year", "month")
    .agg(round(avg("lead_time"), 2).alias("avg_lead_time_days"))
    .orderBy("year", "month")
)
result3.show()

+----+-----+------------------+
|year|month|avg_lead_time_days|
+----+-----+------------------+
|2017|    7|            130.73|
|2017|    8|             35.08|
|2017|    9|             51.72|
|2017|   10|             55.89|
|2017|   11|             33.28|
|2017|   12|             46.75|
|2018|    1|             34.87|
|2018|    2|             30.53|
|2018|    3|             43.19|
|2018|    4|             62.49|
|2018|    5|             60.99|
|2018|    6|             70.64|
|2018|    7|             86.88|
|2018|    8|             83.09|
|2018|    9|             63.32|
|2018|   10|             73.24|
|2018|   11|             44.25|
|2018|   12|             69.75|
+----+-----+------------------+



### Вычислить общую среднюю выручку на каждый месяц в каждом году, сгруппировав по всем типам бронирования для подтвержденных броней, и вывести это в виде сводной таблицы (PIVOT).

In [18]:
confirmed_revenue_df = (logs_hotel_df
    .filter(col("status") == "Not_Canceled")
    .withColumn("total_revenue",
                (col("weekend_nights") + col("week_nights")) * col("avg_room_price"))
    .groupBy("year", "month", "market_segment")
    .agg(round(avg("total_revenue"), 2).alias("avg_revenue"))
)

# Получаем уникальные сегменты рынка для pivot
market_segments = [row["market_segment"] for row in confirmed_revenue_df.select("market_segment").distinct().collect()]

result4_pivot = (confirmed_revenue_df
    .groupBy("year", "month")
    .pivot("market_segment", market_segments)
    .agg(first("avg_revenue"))
    .fillna(0)
    .orderBy("year", "month")
)

result4_pivot.show()

+----+-----+-------------+--------+---------+------+-------+
|year|month|Complementary|Aviation|Corporate|Online|Offline|
+----+-----+-------------+--------+---------+------+-------+
|2017|    7|         22.4|     0.0|   113.75|290.56| 228.95|
|2017|    8|         0.32|     0.0|   156.42|284.21| 235.54|
|2017|    9|        16.89|     0.0|   177.83|348.55| 236.65|
|2017|   10|         1.09|     0.0|   180.26|311.47| 223.24|
|2017|   11|        14.81|     0.0|   102.97|240.52| 198.36|
|2017|   12|         0.25|     0.0|   141.11|258.93| 253.86|
|2018|    1|         2.27|     0.0|   113.03|236.09| 210.51|
|2018|    2|         1.39|   352.0|   115.06|238.07| 251.85|
|2018|    3|        38.17|  118.33|   142.39|301.71| 233.39|
|2018|    4|          0.0|  321.81|   108.42|320.08| 236.44|
|2018|    5|          0.0|   262.5|    229.5|352.34| 274.55|
|2018|    6|          0.0|   247.0|   148.13|335.03| 251.98|
|2018|    7|         5.38|    79.0|   146.99|390.05| 310.36|
|2018|    8|          0.

### Выявить ТОП-5 постоянных гостей, которые принесли наибольшую выручку за все время, и показать их долю в общей выручке от постоянных гостей. Использовать уникальный идентификатор брони как уникальный идентификатор гостя, предположив, что 1 бронь = 1 гость.

In [23]:
repeated_guests_revenue = (logs_hotel_df
    .filter((col("repeated_guest") == 1) & (col("status") == "Not_Canceled"))
    .withColumn("booking_revenue",
                (col("weekend_nights") + col("week_nights")) * col("avg_room_price"))
    .groupBy("ID")
    .agg(round(sum("booking_revenue"), 2).alias("total_revenue"))
)

# Общая выручка от постоянных гостей
total_revenue_result = repeated_guests_revenue.agg(
    round(sum("total_revenue"), 2).alias("overall_revenue")
)
total_revenue_repeated = total_revenue_result.collect()[0]["overall_revenue"]

# ТОП-5 гостей с долей
result5 = (repeated_guests_revenue
    .withColumn("rank", rank().over(Window.orderBy(col("total_revenue").desc())))
    .filter(col("rank") <= 5)
    .withColumn("revenue_percentage",
                round((col("total_revenue") * 100.0 / lit(total_revenue_repeated)), 4))
    .select(col("ID").alias("guest_id"), "total_revenue", "revenue_percentage")
    .orderBy(col("total_revenue").desc())
)

result5.show()

+--------+-------------+------------------+
|guest_id|total_revenue|revenue_percentage|
+--------+-------------+------------------+
|INN19235|       1754.4|              1.55|
|INN05222|        690.0|            0.6096|
|INN14189|        665.0|            0.5875|
|INN09923|        660.0|            0.5831|
|INN25479|        650.0|            0.5743|
+--------+-------------+------------------+

